In [1]:
from requests import get
url = 'https://www.imdb.com/search/title?release_date=2018&sort=num_votes,desc&page=1'
response = get(url)
print(response.text[:500])




<!DOCTYPE html>
<html
    xmlns:og="http://ogp.me/ns#"
    xmlns:fb="http://www.facebook.com/2008/fbml">
    <head>
         
        <meta charset="utf-8">
        <meta http-equiv="X-UA-Compatible" content="IE=edge">

    <meta name="apple-itunes-app" content="app-id=342792525, app-argument=imdb:///?src=mdot">



        <script type="text/javascript">var IMDbTimer={starttime: new Date().getTime(),pt:'java'};</script>

<script>
    if (typeof uet == 'function') {
      uet("bb", "LoadTitle"


In [2]:
from bs4 import BeautifulSoup

html_soup = BeautifulSoup(response.text, 'html.parser')
type(html_soup)

bs4.BeautifulSoup

In [3]:
movie_containers = html_soup.find_all('div', class_ = 'lister-item mode-advanced')
print(type(movie_containers))
print(len(movie_containers))

<class 'bs4.element.ResultSet'>
50


In [4]:
#Extracting the data from one movie first
first_movie = movie_containers[0]
first_movie

<div class="lister-item mode-advanced">
<div class="lister-top-right">
<div class="ribbonize" data-caller="filmosearch" data-tconst="tt4154756"></div>
</div>
<div class="lister-item-image float-left">
<a href="/title/tt4154756/?ref_=adv_li_i"> <img alt="Avengers: Infinity War" class="loadlate" data-tconst="tt4154756" height="98" loadlate="https://m.media-amazon.com/images/M/MV5BMjMxNjY2MDU1OV5BMl5BanBnXkFtZTgwNzY1MTUwNTM@._V1_UX67_CR0,0,67,98_AL_.jpg" src="https://m.media-amazon.com/images/G/01/imdb/images/nopicture/large/film-184890147._CB499613450_.png" width="67"/>
</a> </div>
<div class="lister-item-content">
<h3 class="lister-item-header">
<span class="lister-item-index unbold text-primary">1.</span>
<a href="/title/tt4154756/?ref_=adv_li_tt">Avengers: Infinity War</a>
<span class="lister-item-year text-muted unbold">(2018)</span>
</h3>
<p class="text-muted ">
<span class="certificate">UA</span>
<span class="ghost">|</span>
<span class="runtime">149 min</span>
<span class="ghost">

In [5]:
# Since the content is long so i am again using dev tools to find out movies name and all data
first_movie.div

<div class="lister-top-right">
<div class="ribbonize" data-caller="filmosearch" data-tconst="tt4154756"></div>
</div>

In [6]:
first_movie.a

<a href="/title/tt4154756/?ref_=adv_li_i"> <img alt="Avengers: Infinity War" class="loadlate" data-tconst="tt4154756" height="98" loadlate="https://m.media-amazon.com/images/M/MV5BMjMxNjY2MDU1OV5BMl5BanBnXkFtZTgwNzY1MTUwNTM@._V1_UX67_CR0,0,67,98_AL_.jpg" src="https://m.media-amazon.com/images/G/01/imdb/images/nopicture/large/film-184890147._CB499613450_.png" width="67"/>
</a>

In [7]:
first_movie.h3

<h3 class="lister-item-header">
<span class="lister-item-index unbold text-primary">1.</span>
<a href="/title/tt4154756/?ref_=adv_li_tt">Avengers: Infinity War</a>
<span class="lister-item-year text-muted unbold">(2018)</span>
</h3>

In [8]:
first_movie.h3.a

<a href="/title/tt4154756/?ref_=adv_li_tt">Avengers: Infinity War</a>

In [9]:
first_name = first_movie.h3.a.text
first_name

'Avengers: Infinity War'

In [10]:
# Release year extraction
first_year = first_movie.h3.find('span', class_ = 'lister-item-year text-muted unbold')
first_year

<span class="lister-item-year text-muted unbold">(2018)</span>

In [11]:
first_year = first_year.text
first_year

'(2018)'

In [12]:
# Rating Extraction
first_movie.strong

<strong>8.7</strong>

In [13]:
#Extracting metascore
first_mscore = first_movie.find('span', class_ = 'metascore favorable')

first_mscore = int(first_mscore.text)
print(first_mscore)

68


In [14]:
# Extracting no of votes
first_votes = first_movie.find('span', attrs = {'name':'nv'})
first_votes

<span data-value="402068" name="nv">402,068</span>

In [15]:
first_votes['data-value']

'402068'

In [16]:
first_votes = int(first_votes['data-value'])

In [17]:
eighth_movie_mscore = movie_containers[7].find('div', class_ = 'ratings-metascore')
type(eighth_movie_mscore)

bs4.element.Tag

In [18]:
# Lists to store the scraped data in
names = []
years = []
imdb_ratings = []
metascores = []
votes = []

# Extract data from individual movie container
for container in movie_containers:

    # If the movie has Metascore, then extract:
    if container.find('div', class_ = 'ratings-metascore') is not None:

        # The name
        name = container.h3.a.text
        names.append(name)

        # The year
        year = container.h3.find('span', class_ = 'lister-item-year').text
        years.append(year)

        # The IMDB rating
        imdb = float(container.strong.text)
        imdb_ratings.append(imdb)

        # The Metascore
        m_score = container.find('span', class_ = 'metascore').text
        metascores.append(int(m_score))

        # The number of votes
        vote = container.find('span', attrs = {'name':'nv'})['data-value']
        votes.append(int(vote))

In [19]:
# Now checking the data collected using pandas
import pandas as pd

test_df = pd.DataFrame({'movie': names,
                       'year': years,
                       'imdb': imdb_ratings,
                       'metascore': metascores,
                       'votes': votes})
print(test_df.info())
test_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 5 columns):
imdb         36 non-null float64
metascore    36 non-null int64
movie        36 non-null object
votes        36 non-null int64
year         36 non-null object
dtypes: float64(1), int64(2), object(2)
memory usage: 1.5+ KB
None


,imdb,metascore,movie,votes,year
0,8.7,68,Avengers: Infinity War,402068,(2018)
1,7.5,88,Black Panther,356330,(2018)
2,8.1,66,Deadpool 2,181087,(2018)
3,7.6,64,Ready Player One,176851,(2018)
4,7.8,82,A Quiet Place,167335,(2018)
5,7.0,79,Annihilation,154227,(2018)
6,7.1,62,Solo: A Star Wars Story,110695,(2018)
7,6.4,48,Tomb Raider,108609,(2018)
8,7.0,66,Game Night,96375,(I) (2018)
9,6.5,51,Jurassic World: Fallen Kingdom,89807,(2018)


In [37]:
# Now scraping multiple pages
pages = [str(i) for i in range(1,11)]
years_url = [str(i) for i in range(2018,2019)]

In [23]:
from time import sleep
from random import randint

In [24]:
from time import time

start_time = time()
requests = 0

for _ in range(5):
    # A request would go here
    requests += 1
    sleep(randint(1,3))
    elapsed_time = time() - start_time
    print('Request: {}; Frequency: {} requests/s'.format(requests, requests/elapsed_time))

Request: 1; Frequency: 0.33297048479021046 requests/s
Request: 2; Frequency: 0.3995527505484045 requests/s
Request: 3; Frequency: 0.37468371045295656 requests/s
Request: 4; Frequency: 0.39963923596402867 requests/s
Request: 5; Frequency: 0.4162709203652645 requests/s


In [25]:
from IPython.core.display import clear_output

start_time = time()
requests = 0

for _ in range(5):
    # A request would go here
    requests += 1
    sleep(randint(1,3))
    current_time = time()
    elapsed_time = current_time - start_time
    print('Request: {}; Frequency: {} requests/s'.format(requests, requests/elapsed_time))
    clear_output(wait = True)

Request: 5; Frequency: 0.41579561452873726 requests/s


In [38]:
# Redeclaring the lists to store data in
names = []
years = []
imdb_ratings = []
metascores = []
votes = []

# Preparing the monitoring of the loop
start_time = time()
requests = 0

# For every year in the interval 2000-2017
for year_url in years_url:

    # For every page in the interval 1-4
    for page in pages:

        # Make a get request
        response = get('http://www.imdb.com/search/title?release_date=' + year_url + 
        '&sort=num_votes,desc&page=' + page)#, headers = headers)
        
        # Pause the loop
        sleep(randint(8,15))

        # Monitor the requests
        requests += 1
        elapsed_time = time() - start_time
        print('Request:{}; Frequency: {} requests/s'.format(requests, requests/elapsed_time))
        clear_output(wait = True)

        # Throw a warning for non-200 status codes
        if response.status_code != 200:
            warn('Request: {}; Status code: {}'.format(requests, response.status_code))

        # Break the loop if the number of requests is greater than expected
        if requests > 10:
            warn('Number of requests was greater than expected.')  
            break 

        # Parse the content of the request with BeautifulSoup
        page_html = BeautifulSoup(response.text, 'html.parser')

        # Select all the 50 movie containers from a single page
        mv_containers = page_html.find_all('div', class_ = 'lister-item mode-advanced')

        # For every movie of these 50
        for container in mv_containers:
            # If the movie has a Metascore, then:
            if container.find('div', class_ = 'ratings-metascore') is not None:

                # Scrape the name
                name = container.h3.a.text
                names.append(name)

                # Scrape the year 
                year = container.h3.find('span', class_ = 'lister-item-year').text
                years.append(year)

                # Scrape the IMDB rating
                imdb = float(container.strong.text)
                imdb_ratings.append(imdb)

                # Scrape the Metascore
                m_score = container.find('span', class_ = 'metascore').text
                metascores.append(int(m_score))

                # Scrape the number of votes
                vote = container.find('span', attrs = {'name':'nv'})['data-value']
                votes.append(int(vote))

Request:10; Frequency: 0.05964747914105766 requests/s


In [43]:
test_data = pd.DataFrame({'movie': names,
                              'year': years,
                              'imdb': imdb_ratings,
                              'metascore': metascores,
                              'votes': votes})
print(movie_ratings.info())
movie_ratings.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126 entries, 0 to 125
Data columns (total 6 columns):
movie        126 non-null object
year         126 non-null int64
imdb         126 non-null float64
metascore    126 non-null int64
votes        126 non-null int64
n_imdb       126 non-null float64
dtypes: float64(2), int64(3), object(1)
memory usage: 6.0+ KB
None


,movie,year,imdb,metascore,votes,n_imdb
0,Avengers: Infinity War,2018,8.7,68,402105,87.0
1,Black Panther,2018,7.5,88,356383,75.0
2,Deadpool 2,2018,8.1,66,181127,81.0
3,Ready Player One,2018,7.6,64,176940,76.0
4,A Quiet Place,2018,7.8,82,167482,78.0
5,Annihilation,2018,7.0,79,154263,70.0
6,Solo: A Star Wars Story,2018,7.1,62,110711,71.0
7,Tomb Raider,2018,6.4,48,108651,64.0
8,Game Night,2018,7.0,66,96417,70.0
9,Jurassic World: Fallen Kingdom,2018,6.5,51,89872,65.0


In [44]:
test_data = test_data[['movie', 'year', 'imdb', 'metascore', 'votes']]
test_data.head()

,movie,year,imdb,metascore,votes
0,Avengers: Infinity War,(2018),8.7,68,402105
1,Black Panther,(2018),7.5,88,356383
2,Deadpool 2,(2018),8.1,66,181127
3,Ready Player One,(2018),7.6,64,176940
4,A Quiet Place,(2018),7.8,82,167482


In [45]:
test_data.loc[:, 'year'] = test_data['year'].str[-5:-1].astype(int)

In [46]:
test_data['n_imdb'] = test_data['imdb'] * 10
test_data.head(3)

,movie,year,imdb,metascore,votes,n_imdb
0,Avengers: Infinity War,2018,8.7,68,402105,87.0
1,Black Panther,2018,7.5,88,356383,75.0
2,Deadpool 2,2018,8.1,66,181127,81.0


In [47]:
test_data.to_csv('test_dataset.csv')

In [ ]:
# If you want to increase the test data you only need to increase the value of i at IN[37]